In [35]:
import boto3
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from dotenv import load_dotenv
import os
import pickle
import sklearn  # Añadir esta línea para importar sklearn

print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Sklearn version: {sklearn.__version__}")
print(f"Boto3 version: {boto3.__version__}")


Pandas version: 2.2.3
Numpy version: 2.1.3
Sklearn version: 1.5.2
Boto3 version: 1.35.58


In [36]:
# Configurar la conexión con S3 usando variables de entorno
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('NOTEBOOK_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('NOTEBOOK_ACCESS_KEY_SECRET')
)

bucket_name = 'chicago-inspections-analytics'
ingesta_inicial_key = 'feature-matrix/feature-matrix.2024-11-22.pkl'  # Ruta del archivo en S3

# Función para cargar el archivo de ingesta inicial desde S3
def cargar_feature_matrix():
    try:
        # Descargar el archivo de S3
        file_obj = s3_client.get_object(Bucket=bucket_name, Key=ingesta_inicial_key)
        file_data = file_obj['Body'].read()
        
        # Convertir el archivo de Pickle a un DataFrame
        df = pickle.loads(file_data)
        print("matriz de caracteristicas cargada exitosamente.")
        return df
    except Exception as e:
        print("Error al cargar la matriza de caracteristicas desde S3:", e)
        return None

# Cargar el DataFrame de ingesta inicial
df_inicial = cargar_feature_matrix()

# Obtener la fecha más reciente
if df_inicial is not None:
   df_inicial.head()
else:
    print("No se pudo cargar la matriz de carateriscticas.")

matriz de caracteristicas cargada exitosamente.


In [38]:
df_inicial.head()

,facility_type__facility_type_BANQUET HALL,facility_type__facility_type_Bakery,facility_type__facility_type_Catering,facility_type__facility_type_Children's Services Facility,facility_type__facility_type_GAS STATION,facility_type__facility_type_Golden Diner,facility_type__facility_type_Grocery Store,facility_type__facility_type_Hospital,facility_type__facility_type_Liquor,facility_type__facility_type_Long Term Care,...,remainder__latitude,remainder__longitude,remainder__results,remainder__month,remainder__year,remainder__day_of_month,remainder__week_of_year,remainder__week_day,remainder__weekend,remainder__day_of_week
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.884586,-87.63101,pass,1,2010,4,1,1,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,pass,1,2010,4,1,1,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,pass,1,2010,4,1,1,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.898431,-87.628009,fail,1,2010,4,1,1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.938443,-87.768318,pass,1,2010,4,1,1,0,0


In [39]:
# Verificar si las columnas necesarias existen
columnas_fecha = ['remainder__year', 'remainder__month', 'remainder__day_of_month']
if set(columnas_fecha).issubset(df_inicial.columns):

    # Convertir las columnas a tipo numérico
    df_inicial[columnas_fecha] = df_inicial[columnas_fecha].apply(pd.to_numeric, errors='coerce')

    # Ordenar por año, mes y día de más antiguo a más reciente
    df_inicial = df_inicial.sort_values(by=['remainder__year', 'remainder__month', 'remainder__day_of_month'])

    # Separar características (X) y variable objetivo (y)
    X = df_inicial.drop(columns=['remainder__results'])  # Eliminar la variable objetivo
    y = df_inicial['remainder__results']  # Variable objetivo

    # Dividir en 70% entrenamiento y 30% prueba sin aleatorización, usando la semilla
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20201122, shuffle=False)

    print("Datos separados correctamente y ordenados.")
else:
    print("Error: No se encontraron todas las columnas necesarias para ordenar los datos.")


Datos separados correctamente y ordenados.


In [41]:
# Configurar la conexión con S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('NOTEBOOK_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('NOTEBOOK_ACCESS_KEY_SECRET')
)

# Función para guardar un archivo en S3
def guardar_en_s3(data, bucket_name, path):
    try:
        # Convertir el dataframe a pickle
        pickle_data = pickle.dumps(data)

        # Subir el archivo al bucket S3
        s3_client.put_object(Bucket=bucket_name, Key=path, Body=pickle_data)
        print(f"Archivo guardado correctamente en {bucket_name}/{path}")
    except Exception as e:
        print(f"Error al guardar el archivo en S3: {e}")

# Guardar el conjunto de entrenamiento
train_path = 'dataset/train/train_dataset.pkl'
guardar_en_s3(X_train, bucket_name, train_path)

# Guardar el conjunto de pruebas
test_path = 'dataset/test/test_dataset.pkl'
guardar_en_s3(X_test, bucket_name,test_path)

Archivo guardado correctamente en chicago-inspections-analytics/dataset/train/train_dataset.pkl
Archivo guardado correctamente en chicago-inspections-analytics/dataset/test/test_dataset.pkl


In [45]:
# Definir el modelo base de Árbol de Decisión
dt = DecisionTreeClassifier()

# Definir la cuadrícula de hiperparámetros
param_grid = {
    'max_depth': [5, 7, 9,11,15],  # Profundidad máxima del árbol
    'min_samples_leaf': [5,7,9,11,13],  # Mínimo de muestras por hoja
    'criterion': ['gini', 'entropy']  # Criterio de ganancia de información
}

# Configuración de TimeSeriesSplit (5 pliegues)
tscv = TimeSeriesSplit(n_splits=5)

# Configurar GridSearchCV con precisión como métrica de evaluación
grid_search = GridSearchCV(
    estimator=dt, 
    param_grid=param_grid, 
    cv=tscv, 
    scoring='precision',  # Métrica de precisión
    n_jobs=-1, 
    verbose=1
)

# Entrenar el modelo con GridSearchCV
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Obtener el mejor modelo
best_dt_model = grid_search.best_estimator_

# Hacer predicciones con el mejor modelo
y_pred = best_dt_model.predict(X_test)

# Calcular la precisión en el conjunto de prueba
print(f"Precisión en conjunto de prueba: {precision_score(y_test, y_pred, pos_label='pass')}")

# Función para guardar el modelo en un bucket de S3 usando pickle
def save_model(model, bucket_name, model_name):
    # Guardar el modelo localmente como un archivo pickle
    model_filename = f"{model_name}.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    # Subir el archivo pickle al bucket de S3
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('NOTEBOOK_ACCESS_KEY'),
        aws_secret_access_key=os.getenv('NOTEBOOK_ACCESS_KEY_SECRET')
    )

    s3_client.upload_file(model_filename, bucket_name, f"modelos/{model_filename}")
    print(f"Modelo guardado en S3 en: {bucket_name}/modelos/{model_filename}")
save_model(best_dt_model, bucket_name, "arboles_gridsearch")


Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\PC\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
C:\Users\PC\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Mejores parámetros encontrados: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}
Precisión en conjunto de prueba: 0.7481371777664646
Modelo guardado en S3 en: chicago-inspections-analytics/modelos/arboles_gridsearch.pkl


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import precision_score, make_scorer

# Definir el modelo base
rf = RandomForestClassifier()

# Definir los hiperparámetros a ajustar
param_grid = {
    'max_depth': [5, 7, 9, 13],  # Opciones de profundidad máxima
    'min_samples_leaf': [5, 9, 11],  # Opciones de cantidad mínima de muestras por hoja
    'n_estimators': [300, 500, 800, 1000, 1300]  # Opciones de número de estimadores
}

# Configuración de TimeSeriesSplit (5 pliegues)
tscv = TimeSeriesSplit(n_splits=5)

# Usar make_scorer para especificar 'precision' como métrica de evaluación
precision_scorer = make_scorer(precision_score, pos_label='pass')

# Configuración de GridSearchCV con TimeSeriesSplit y precisión como métrica de evaluación
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=tscv,
    scoring=precision_scorer,  # Usar precisión como métrica de selección
    n_jobs=-1,  # Ejecutar en un solo hilo
    verbose=1
)

# Entrenar el modelo con GridSearchCV
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Obtener el mejor modelo
best_rf_model = grid_search.best_estimator_

# Hacer predicciones con el mejor modelo
y_pred = best_rf_model.predict(X_test)

# Imprimir precisión en el conjunto de prueba
print(f"Precisión en conjunto de prueba: {precision_score(y_test, y_pred, pos_label='pass')}")

# Guardar el modelo entrenado en el bucket de S3 (si es necesario)
save_model(best_rf_model, bucket_name, "Random_forest")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

In [44]:
# Definir el modelo base de regresión logística
log_reg = LogisticRegression(solver='liblinear')  # El solver 'liblinear' es adecuado para penalización L1

# Configuración de la búsqueda de cuadrícula con los hiperparámetros que mencionas
param_grid = {
    'penalty': ['l2', 'none'],  # Penalización L2 o sin penalización
    'max_iter': [300, 500, 1000],  # Número de iteraciones
    'solver': ['liblinear']  # Solver adecuado para las penalizaciones L1 y L2
}

# Configuración de TimeSeriesSplit (5 pliegues)
tscv = TimeSeriesSplit(n_splits=5)

# Configuración de GridSearchCV con precisión como la métrica de evaluación
precision_scorer = make_scorer(precision_score, pos_label='pass')

grid_search = GridSearchCV(
    estimator=log_reg, 
    param_grid=param_grid, 
    cv=tscv, 
    scoring=precision_scorer,  # Métrica de precisión
    n_jobs=-1, 
    verbose=1
)

# Entrenar el modelo con GridSearchCV (usando el conjunto de entrenamiento)
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Obtener el mejor modelo
best_log_reg_model = grid_search.best_estimator_

# Hacer predicciones con el mejor modelo
y_pred = best_log_reg_model.predict(X_test)

# Imprimir precisión en el conjunto de prueba
# Cambiar el valor de pos_label a 'pass'
print(f"Precisión en conjunto de prueba: {precision_score(y_test, y_pred, pos_label='pass')}")

# Guardar el modelo entrenado en el bucket de S3
save_model(best_log_reg_model, bucket_name, 'logisticregression_gridsearch')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 